In [3]:
import cv2
import PIL
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import tqdm
import os
import random

from PIL import Image
from torchvision import transforms as T
from torchvision.models.alexnet import AlexNet
from torchvision.models.resnet import resnet18, resnet34, resnet50, resnet101, resnet152
from torch.utils.data import IterableDataset, DataLoader, Dataset
from sklearn.metrics import classification_report, accuracy_score, f1_score, average_precision_score
from sklearn.model_selection import train_test_split, KFold

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'


def linear_combination(x, y, epsilon): 
    return epsilon*x + (1-epsilon)*y


def reduce_loss(loss, reduction='mean'):
    return loss.mean() if reduction=='mean' else loss.sum() if reduction=='sum' else loss


class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, epsilon:float=0.1, reduction='mean'):
        super().__init__()
        self.epsilon = epsilon
        self.reduction = reduction
    
    def forward(self, preds, target):
        n = preds.size()[-1]
        log_preds = F.log_softmax(preds, dim=-1)
        loss = reduce_loss(-log_preds.sum(dim=-1), self.reduction)
        nll = F.nll_loss(log_preds, target, reduction=self.reduction)
        return linear_combination(loss/n, nll, self.epsilon)
    
    
class BonzTrainDataset(Dataset):
    def __init__(self, data):
        super(BonzTrainDataset, self).__init__()
        self.image_prefixes = data.image_prefixes.values
        self.features = data.features.values
        self.img_tensors = data.img_tensors.values
        if 'labels' in data:
            self.labels = data.labels.values
            self.one_hot_labels = data.one_hot_labels.values
        else:
            self.labels=None
    
    def __len__(self):
        return 3000
    
    def __getitem__(self, idx):
        ''' Full data
        x1 = idx // len(self.image_prefixes)
        x2 = idx % len(self.image_prefixes)
        '''
        
        # Random select
        x1 = random.randint(0, len(self.image_prefixes)-1)
        x2 = random.randint(0, len(self.image_prefixes)-1)
        
        if idx%3 == 0:
            while self.labels[x1] != self.labels[x2]:
                x2 = random.randint(0, len(self.image_prefixes)-1)
        else:
            while self.labels[x1] == self.labels[x2]:
                x2 = random.randint(0, len(self.image_prefixes)-1)
        
        outputs = (self.img_tensors[x1], 
                   self.features[x1], 
                   self.img_tensors[x2], 
                   self.features[x2],)
        if self.labels[x1] == self.labels[x2]:
            outputs += (torch.tensor([1]),)
        else:
            outputs += (torch.tensor([0]),)
        return outputs
    
        
class Bonz(nn.Module):
    def __init__(self, hidden_dim=100, feature_selection=12):
        super(Bonz, self).__init__()
        self.resnet = resnet50(True)
        self.BiLSTM = nn.LSTM(2048, hidden_dim, num_layers=2, batch_first=True, bidirectional=True)
        self.bn = nn.BatchNorm1d(hidden_dim*2 + feature_selection)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim*2 + feature_selection, hidden_dim*2 + feature_selection),
            nn.LeakyReLU(0.001, inplace=True),
            nn.Dropout(0.1),
            nn.Linear(hidden_dim*2 + feature_selection, 1)
        )
        self.classifier.apply(self._init_weights)
        self.classifier.apply(self._xavier)
        self.BiLSTM.apply(self._xavier)
    
    def forward(self, x1, x1_f, x2, x2_f):
        ''' PROCESS X1'''
        # Generate featuress from each images
        x = []
        for img in x1:
            temp = self.do_resnet(img)
            x.append(temp.unsqueeze(0))
        x = torch.cat(x, 0)
        
        # LSTM step
        x, _ = self.BiLSTM(x)
        lstm_features = x[:,-1,:]
        
        # Concate features
        x1 = torch.cat([lstm_features, x1_f], -1)
        #x1 = self.bn(x1)
        
        ''' PROCESS X2'''
        # Generate featuress from each images
        x = []
        for img in x2:
            temp = self.do_resnet(img)
            x.append(temp.unsqueeze(0))
        x = torch.cat(x, 0)
        
        # LSTM step
        x, _ = self.BiLSTM(x)
        lstm_features = x[:,-1,:]
        
        # Concate features
        x2 = torch.cat([lstm_features, x2_f], -1)
        #x2 = self.bn(x2)
        
        ''' DIFFERENCE BETWEEN X1 and X2'''
        dif = torch.abs(x1-x2)   
        
        predict = self.classifier(dif)

        return (predict, dif,)
    
    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding, nn.Conv1d)):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if isinstance(module, (nn.Linear, nn.Conv1d)) and module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
            
    def _xavier(self, module):
        for name, param in module.named_parameters():
            if 'weight' in name:
                nn.init.xavier_normal_(param)
            elif 'bias' in name:
                param.data.zero_()
    
    def do_resnet(self, img):
        x = self.resnet.conv1(img)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)

        x = self.resnet.layer1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)

        x = self.resnet.avgpool(x)
        x = torch.flatten(x, 1)
        
        return x
    
    def freeze_resnet(self):
        for w in self.resnet.parameters():
            w.requires_grad = False
            
    def unfreeze_resnet(self):
        for w in self.resnet.parameters():
            w.requires_grad = True 

In [4]:
def get_data(path):
    data = pd.read_csv(path)
    resnet_idx = list(data.columns).index('data_AUTOGRAPHER_RESNET_mean_tench, Tinca tinca')
    new_data = data.iloc[:,:resnet_idx]
    new_data = new_data.drop([col_len for col_len in new_data.keys() if '_len' in col_len], 1) # Drop columns with _LEN
    new_data['labels'] = [int(i[-2:])-1 for i in new_data.event_id.values]
    new_data['image_prefixes'] = list(map(lambda x,y,z: str(x)+'_'+str(y)+'_'+str(z), 
                                      new_data.sub_id.values, 
                                      new_data.source.values, 
                                      new_data.event_id.values))
    new_data['one_hot_labels'] = list(map(lambda x: nn.functional.one_hot(torch.tensor(x), 20).float(), new_data.labels.values))
    new_data['features'] = [torch.tensor(i).float() for i in new_data.iloc[:, 3:-3].values]
    new_data['img_tensors'] = get_img_tensors(new_data['image_prefixes'].values)
    return new_data.iloc[:, -5:]


def get_test_data(path):
    data = pd.read_csv(path, index_col=0)
    resnet_idx = list(data.columns).index('data_AUTOGRAPHER_RESNET_mean_tench, Tinca tinca')
    new_data = data.iloc[:,:resnet_idx]
    new_data = new_data.drop([col_len for col_len in new_data.keys() if '_len' in col_len], 1) # Drop columns with _LEN
    new_data['image_prefixes'] = list(map(lambda x,y: str(x)+'_pred'+str(y), 
                                      new_data.sub_id.values, 
                                      new_data.event_id.values))
    new_data['features'] = [torch.tensor(i).float() for i in new_data.iloc[:, 3:-1].values]
    new_data['img_tensors'] = get_img_tensors(new_data['image_prefixes'].values)
    return new_data.iloc[:, [0,1,2,-3,-2,-1]]


def get_img_tensors(image_prefixes):
    transform = T.Compose([
        T.Resize(256),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    temp = []
    
    for img_prefix in tqdm.notebook.tqdm(image_prefixes):
        img_paths = []
        i = 0
        img_name = img_prefix+'_'+str(i)+'.jpg'
        while img_name in os.listdir('./OUTPUT_MERGED/AUTOGRAPHER/'):
            img_paths.append('./OUTPUT_MERGED/AUTOGRAPHER/'+img_name)
            i += 1
            img_name = img_prefix+'_'+str(i)+'.jpg'

        # Transform images to tensors
        img_tensors = []
        for path in img_paths:
            img = Image.open(path)
            img_tensors.append(transform(img))
            
        # padding img tensors
        if len(img_tensors) < 16:
            dump = torch.zeros((3,224,224)).float()
            dump = [dump] * (16 - len(img_tensors))
            img_tensors.extend(dump)
            
        temp.append(torch.stack(img_tensors,0))
    
    return temp

def check_params(model):
    model.freeze_resnet()
    print(sum([i.numel() for i in model.parameters() if i.requires_grad]))
    model.unfreeze_resnet()
    print(sum([i.numel() for i in model.parameters() if i.requires_grad]))
    

        
train_data = get_data('train_min_max.csv')
#train_data = train_data.sort_values(by='image_prefixes', ignore_index=True)
test_data = get_test_data('test_min_max.csv')

# Init MODEL

In [5]:
model = Bonz(hidden_dim=128, feature_selection=train_data.features[0].shape[0])
check_params(model)

2746637
28303669


# Create Dataset & Dataloader

In [6]:
dataset = BonzTrainDataset(train_data)
dataloader = DataLoader(dataset, batch_size=2, shuffle=False)

dataset.__len__()

3000

# Find Best Learning Rate

In [15]:
model.freeze_resnet()

model.to(DEVICE)
#torch.save(model.state_dict(), 'origin_sd.pt')
origin_state_dict = torch.load('origin_sd.pt')
model.train()

start_lr = 1e-3
lr_find_epochs = 5

dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), start_lr)
criterion = nn.BCEWithLogitsLoss()



# Make lists to capture the logs
lr_find_acc = []
lr_find_loss = []
lr_find_lr = []


for _ in range(2):
    scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, 
                                                  base_lr=0, 
                                                  max_lr=start_lr*10, 
                                                  step_size_up=lr_find_epochs,
                                                  cycle_momentum=False)
    scheduler.step()
    
    for i in tqdm.notebook.trange(lr_find_epochs):
        # Load origin state dict
        model.load_state_dict(origin_state_dict)

        predicts = []
        y_true = []
        total_loss = 0

        for x1, x1_f, x2, x2_f, label in tqdm.notebook.tqdm(dataloader, desc='Training: '):

            x1 = [ts.to(DEVICE) for ts in x1]
            x1_f = x1_f.to(DEVICE)
            x2 = [ts.to(DEVICE) for ts in x2]
            x2_f = x2_f.to(DEVICE)
            label = label.to(DEVICE)

            optimizer.zero_grad()

            predict = model(x1, x1_f, x2, x2_f)[0]
            loss = criterion(predict, label.float())

            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            predict = predict.detach().cpu()
            predict = torch.sigmoid(predict)
            predicts.extend(predict.tolist())
            y_true.extend(label.detach().cpu().tolist())

        train_acc = accuracy_score(np.array(y_true), np.array(predicts)>0.5)
        lr_step = optimizer.state_dict()["param_groups"][0]["lr"]
        print(f'epoch={i}, Acc={train_acc*100:.2f}, Loss={loss:.2f}, LR={lr_step:.2e}')

        lr_find_lr.append(lr_step)
        lr_find_acc.append(train_acc)
        lr_find_loss.append(total_loss)

        scheduler.step()
    
    start_lr *= 10
        
        
for a, b, c in zip(lr_find_lr, lr_find_acc, lr_find_loss):
    print(f'{a}\t{b}\t{c}')


epoch=0, Acc=71.63, Loss=0.48, LR=2.00e-03



epoch=1, Acc=75.27, Loss=0.39, LR=4.00e-03



epoch=2, Acc=73.57, Loss=0.39, LR=6.00e-03



epoch=3, Acc=75.20, Loss=0.60, LR=8.00e-03



epoch=4, Acc=74.83, Loss=0.48, LR=1.00e-02




epoch=0, Acc=75.93, Loss=0.34, LR=2.00e-02



epoch=1, Acc=76.33, Loss=0.47, LR=4.00e-02



epoch=2, Acc=74.03, Loss=0.49, LR=6.00e-02



epoch=3, Acc=75.27, Loss=0.43, LR=8.00e-02



epoch=4, Acc=70.33, Loss=0.52, LR=1.00e-01

0.0020000000000000018	0.7163333333333334	24.896021991968155
0.003999999999999999	0.7526666666666667	22.936850368976593
0.006000000000000001	0.7356666666666667	23.772990942001343
0.007999999999999998	0.752	23.320587068796158
0.01	0.7483333333333333	23.09900239109993
0.020000000000000018	0.7593333333333333	22.365572661161423
0.039999999999999994	0.7633333333333333	22.559796035289764
0.06000000000000001	0.7403333333333333	24.269256711006165
0.07999999999999999	0.7526666666666667	24.01339900493622
0.1	0.7033333333333334	28.055756986141205


In [17]:
model.freeze_resnet()

model.to(DEVICE)
#torch.save(model.state_dict(), 'origin_sd.pt')
origin_state_dict = torch.load('origin_sd.pt')
model.train()

start_lr = 2e-2
lr_find_epochs = 20

dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), start_lr)
criterion = nn.BCEWithLogitsLoss()


# Make lists to capture the logs
best_metrics = {'train_loss': 1e10, 
                'train_acc': 0}

metrics = {'train_loss': [], 
           'train_acc': []}

    
for e in tqdm.notebook.trange(lr_find_epochs):
    # Load origin state dict
    model.load_state_dict(origin_state_dict)

    predicts = []
    y_true = []
    total_loss = 0

    for x1, x1_f, x2, x2_f, label in dataloader:

        x1 = [ts.to(DEVICE) for ts in x1]
        x1_f = x1_f.to(DEVICE)
        x2 = [ts.to(DEVICE) for ts in x2]
        x2_f = x2_f.to(DEVICE)
        label = label.to(DEVICE)

        optimizer.zero_grad()

        predict = model(x1, x1_f, x2, x2_f)[0]
        loss = criterion(predict, label.float())

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        predict = predict.detach().cpu()
        predict = torch.sigmoid(predict)
        predicts.extend(predict.tolist())
        y_true.extend(label.detach().cpu().tolist())

    train_acc = accuracy_score(np.array(y_true), np.array(predicts)>0.5)
    print(f'epoch={e}, Acc={train_acc*100:.2f}, Loss={loss:.2f}')

    metrics['train_acc'].append(train_acc)
    metrics['train_loss'].append(total_loss)
    
    if total_loss < best_metrics['train_loss']:
        best_metrics['train_acc'] = train_acc
        best_metrics['train_loss'] = total_loss
        
        torch.save(model.state_dict(), 'best_model.pt')
        print(f'BEST MODEL at Epoch = {e}')

print(metrics)

epoch=0, Acc=72.17, Loss=0.50
BEST MODEL at Epoch = 0
epoch=0, Acc=75.33, Loss=0.55
BEST MODEL at Epoch = 1
epoch=0, Acc=75.50, Loss=0.44
BEST MODEL at Epoch = 2
epoch=0, Acc=76.40, Loss=0.35
BEST MODEL at Epoch = 3
epoch=0, Acc=75.87, Loss=0.36
epoch=0, Acc=75.00, Loss=0.41
epoch=0, Acc=74.83, Loss=0.41
epoch=0, Acc=75.67, Loss=0.34
epoch=0, Acc=76.23, Loss=0.39
epoch=0, Acc=76.10, Loss=0.47
epoch=0, Acc=74.23, Loss=0.50
epoch=0, Acc=74.43, Loss=0.44
epoch=0, Acc=74.00, Loss=0.46
epoch=0, Acc=74.27, Loss=0.46
epoch=0, Acc=74.67, Loss=0.51
epoch=0, Acc=75.10, Loss=0.41
epoch=0, Acc=75.97, Loss=0.39
epoch=0, Acc=75.07, Loss=0.35
epoch=0, Acc=74.80, Loss=0.41
epoch=0, Acc=74.53, Loss=0.50

{'train_loss': [26.103297501802444, 23.226665496826172, 22.845528066158295, 22.22298690676689, 22.338309109210968, 22.902508944272995, 23.020601972937584, 22.90515437722206, 22.793964117765427, 23.096015751361847, 23.716843843460083, 23.15110620856285, 23.965454638004303, 23.386887043714523, 23.9242077

In [30]:
accuracy_score(np.array(y_true).squeeze(), (np.array(predicts)>0.5).squeeze())

0.7453333333333333

In [31]:
np.array(predicts).squeeze()[-10:]

array([1.17710277e-01, 6.68731630e-01, 2.39934889e-03, 5.80272079e-01,
       2.56267581e-02, 6.34546997e-03, 4.51944172e-01, 9.04706836e-01,
       6.41793071e-04, 2.21925691e-01])

In [19]:
MAX_LR = 0.065
MIN_LR = 0.025
NUM_EPOCH = 150


# Split data
train_dataset = train_data.loc[140:]
val_dataset = train_data.loc[:139]

train_dataset = BonzDataset(train_dataset)
val_dataset = BonzDataset(val_dataset)

data_loader = DataLoader(train_dataset, batch_size=40, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=40)

best_metrics = {'train_loss': 1e10, 
                'val_loss': 1e10, 
                'train_acc': 0, 
                'val_acc': 0}

''' TRAINING WITH FREEZE ResNet
'''
model.to(DEVICE)
torch.save(model.state_dict(), 'origin_sd.pt')
origin_state_dict = torch.load('origin_sd.pt')

model.freeze_resnet()
model.load_state_dict(origin_state_dict) #Load origin state dict

loss_fn = nn.CrossEntropyLoss()
#optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001, betas=(0.9, 0.99))
optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=MAX_LR)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=10, verbose=True)
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, 
                                              base_lr=MIN_LR, 
                                              max_lr=MAX_LR, 
                                              step_size_up=(NUM_EPOCH//8)*len(data_loader))

for e in tqdm.notebook.trange(NUM_EPOCH):
    
    # Training Phase
    model.train()
    s = time.time()
    train_loss = 0
    predicts = []
    y_true = []
    
    for img_tensors, features, one_hot_label, label in data_loader:
        img_tensors = [ts.to(DEVICE) for ts in img_tensors]
        features = features.to(DEVICE)
        one_hot_label = one_hot_label.to(DEVICE)
        label = label.to(DEVICE)

        optimizer.zero_grad()
        predict = model(img_tensors, features)
        loss = loss_fn(predict, label)
        loss.backward()
        optimizer.step()
        if e >= 19:
            scheduler.step()

        train_loss += loss.item()
        predict = predict.detach().cpu()
        predict = torch.argmax(predict, 1)
        predicts.extend(predict.tolist())
        y_true.extend(label.detach().cpu().tolist())

    train_acc = accuracy_score(y_true, predicts)
    lr_step = optimizer.state_dict()["param_groups"][0]["lr"]

    # Validation Phase
    model.eval()
    val_loss = 0
    predicts = []
    y_true = []
    
    for img_tensors, features, one_hot_label, label in val_data_loader:
        img_tensors = [ts.to(DEVICE) for ts in img_tensors]
        features = features.to(DEVICE)
        one_hot_label = one_hot_label.to(DEVICE)
        label = label.to(DEVICE)

        with torch.no_grad():
            predict = model(img_tensors, features)
            loss = loss_fn(predict, label)
            val_loss += loss.item()
        predict = predict.detach().cpu()
        predict = torch.argmax(predict, 1)
        predicts.extend(predict.tolist())
        y_true.extend(label.detach().cpu().tolist())

    val_acc = accuracy_score(y_true, predicts)

    # Scheduler step wise
    #scheduler.step(0.25*train_loss + 0.75*val_loss)
    
    print(f'Epoch={e}, Train_loss={train_loss:.1f}, Val_loss={val_loss:.1f}, Train_acc={train_acc*100:.2f}, Val_acc={val_acc*100:.2f}, LR={lr_step:.2e}, , Time={time.time()-s:.2f}')
    
    if val_loss < best_metrics['val_loss']:
        best_metrics['train_acc'] = train_acc
        best_metrics['val_acc'] = val_acc
        best_metrics['train_loss'] = train_loss
        best_metrics['val_loss'] = val_loss

        torch.save(model.state_dict(), 'best_model.pt')
        print(f'BEST MODEL at Epoch = {e}')

Epoch=0, Train_loss=12.0, Val_loss=12.0, Train_acc=5.71, Val_acc=8.57, LR=2.50e-02, , Time=8.76
BEST MODEL at Epoch = 0
Epoch=1, Train_loss=12.0, Val_loss=12.0, Train_acc=5.71, Val_acc=6.43, LR=2.50e-02, , Time=8.68
BEST MODEL at Epoch = 1
Epoch=2, Train_loss=12.0, Val_loss=11.9, Train_acc=11.43, Val_acc=10.00, LR=2.50e-02, , Time=8.72
BEST MODEL at Epoch = 2
Epoch=3, Train_loss=11.9, Val_loss=11.9, Train_acc=10.71, Val_acc=7.86, LR=2.50e-02, , Time=8.73
BEST MODEL at Epoch = 3
Epoch=4, Train_loss=11.9, Val_loss=11.9, Train_acc=6.43, Val_acc=5.00, LR=2.50e-02, , Time=8.77
BEST MODEL at Epoch = 4
Epoch=5, Train_loss=11.9, Val_loss=11.9, Train_acc=6.43, Val_acc=6.43, LR=2.50e-02, , Time=9.06
BEST MODEL at Epoch = 5
Epoch=6, Train_loss=11.8, Val_loss=11.8, Train_acc=10.00, Val_acc=13.57, LR=2.50e-02, , Time=8.80
BEST MODEL at Epoch = 6
Epoch=7, Train_loss=11.8, Val_loss=11.8, Train_acc=13.57, Val_acc=17.14, LR=2.50e-02, , Time=8.83
BEST MODEL at Epoch = 7
Epoch=8, Train_loss=11.8, Val_los

In [20]:
model.eval()
#model.load_state_dict(torch.load('best_model.pt'))
predict_matrix = []
val_loss = 0
val_data_loader = DataLoader(BonzDataset(train_data), batch_size=32)
for img_tensors, features, one_hot_label, label in val_data_loader:
    img_tensors = [ts.to(DEVICE) for ts in img_tensors]
    features = features.to(DEVICE)
    one_hot_label = one_hot_label.to(DEVICE)
    label = label.to(DEVICE)

    with torch.no_grad():
        predict = model(img_tensors, features)
    predict = predict.detach().cpu()
    predict_matrix.extend(torch.softmax(predict, 1))
    
onehot_true = np.array(list(i.numpy() for i in train_data.one_hot_labels.values))
onehot_predict = torch.stack(predict_matrix).numpy()

average_precision_score(onehot_true, onehot_predict)

0.8264700613043315

In [18]:
np.argmax(onehot_predict, 1)

array([11, 11,  7,  3,  5,  5,  6,  7,  6,  9, 14, 11, 12, 13, 14,  1, 16,
       17, 18, 19, 10,  1,  1,  3,  3,  9,  8, 14,  5, 10, 10,  1, 18, 13,
       13, 10, 16, 17, 18, 19,  2,  1,  0,  2,  4,  5,  6,  7,  8, 14, 15,
       10, 15, 13, 14,  1, 19, 19, 18, 19,  0, 11, 10, 10, 11,  9,  6, 11,
        5,  5,  1, 11, 12, 14, 12, 15, 12, 19, 18, 19,  0, 10,  0,  7, 16,
        7,  6,  7,  8,  9, 10, 10,  6, 13, 14,  1, 16, 17, 18, 19,  0, 15,
        0,  3, 10,  5, 16,  7,  6,  9, 10, 15, 12, 13, 11,  3, 16, 17, 18,
       19,  0,  1,  4, 11,  4,  5,  6,  7,  5,  5,  1,  1, 12, 13, 14, 15,
       17, 19, 18, 19,  2,  1,  1,  3,  5,  7,  6,  7,  8,  9, 10,  5, 12,
       13, 14, 15, 16, 17, 18, 19,  0,  1,  2,  3, 11,  5,  8,  7,  8,  9,
       10,  1, 12, 13, 14, 15, 16, 17, 18, 19,  0,  1,  2,  3,  4,  0,  6,
        7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 11,  1,  1, 15,
        4,  5,  6,  7,  8,  9, 10,  1, 12, 13, 14, 15, 16, 17, 18, 19,  0,
        1,  2,  3, 11,  5

In [21]:
model.eval()
predict_matrix = []
val_data_loader = DataLoader(BonzDataset(test_data), batch_size=32)
for img_tensors, features in val_data_loader:
    img_tensors = [ts.to(DEVICE) for ts in img_tensors]
    features = features.to(DEVICE)
    with torch.no_grad():
        predict = model(img_tensors, features)
    predict = predict.detach().cpu()
    predict_matrix.extend(torch.softmax(predict, 1))

onehot_predict = torch.stack(predict_matrix).numpy()

COLUMN_NAMES = ['act'+str(i)+str(j) for i in range(2) for j in range(10)]
COLUMN_NAMES.pop(0)
COLUMN_NAMES.append('act20')

df_prob = pd.DataFrame(data=onehot_predict, columns=COLUMN_NAMES)
df_prob['event'] = test_data['image_prefixes']

submission = []
for act in list(df_prob.keys())[:-1]:
    ranked_ = df_prob.sort_values(by=[act], ascending=False)['event'].values
    ranked_ = act+' '+ranked_
    submission.extend(ranked_)

pd.DataFrame(submission).to_csv('submission.txt', header=False, index=False)

np.argmax(onehot_predict, 1)

array([10, 10, 17, 17,  3, 14,  7,  3,  7,  3, 11, 14,  3, 14, 14, 14,  3,
       13, 14,  3,  3, 14,  3, 14,  3, 14, 14,  3, 14, 14,  3,  3, 14, 14,
        3,  3, 14, 14,  3,  3,  3,  6, 13, 13, 14, 14,  3,  3,  3, 14,  3,
       14, 13, 14,  3, 14,  3, 14,  3,  3,  3,  3,  3, 14, 14, 14,  3,  7,
        3,  3, 14, 14, 13,  3, 14, 14, 14, 14, 14,  3, 14, 14,  3, 14,  3,
       14,  3, 14, 14,  3, 13, 14, 14,  3,  3, 14, 14, 14, 14,  3,  3,  3,
        3, 14, 14, 14, 14, 14, 14, 13,  3,  3, 14,  3,  6, 14,  3, 16, 14,
        3, 11, 13, 14,  7, 14,  3, 14,  3,  3, 14, 14, 14,  3,  7, 14,  3,
        3,  3, 14, 14], dtype=int64)